# Model pipeline
Let's start with our prepared dataset:

In [1]:
import pandas as pd
weather_train_scaled = pd.read_csv('data/weather_train_scaled.csv')

To define a classifier, we need to say what is the input data (`X`) and what is the target (`y`).
For now, we focus on only the numerical features

In [2]:
features = weather_train_scaled.columns[1:]
print(features)

Index(['BASEL_cloud_cover', 'BASEL_humidity', 'BASEL_pressure',
       'BASEL_global_radiation', 'BASEL_precipitation', 'BASEL_sunshine',
       'BASEL_temp_mean', 'BASEL_temp_min', 'BASEL_temp_max',
       'BUDAPEST_cloud_cover',
       ...
       'STOCKHOLM_temp_min', 'STOCKHOLM_temp_max', 'TOURS_wind_speed',
       'TOURS_humidity', 'TOURS_pressure', 'TOURS_global_radiation',
       'TOURS_precipitation', 'TOURS_temp_mean', 'TOURS_temp_min',
       'TOURS_temp_max'],
      dtype='object', length=163)


In [3]:
X = weather_train_scaled[features]
y = weather_train_scaled['MONTH']

We try out one of the simplest classification: a k-nearest neighbor classifier. To classify an examlpe, this model looks at the `k` neirest neighbors in the training set and chooses the majority class.

In [4]:
from sklearn.neighbors import KNeighborsClassifier

In [5]:
# First we define the model and its parameters, in this case the number of neighbors
classifier = KNeighborsClassifier(n_neighbors=3)
# Now we train it on the data
classifier.fit(X, y)

KNeighborsClassifier(n_neighbors=3)

We can now use our classifier to make predictions:

In [6]:
predictions = classifier.predict(X)
predictions[:10]

array([ 6,  3,  3,  1,  5,  3,  2,  1,  6, 11])

and compare this with the true target labels, for example using a confusion matrix. The rows in the confusion matrix 
are the true labels and the columns are the predicted labels

In [22]:
from sklearn.metrics import confusion_matrix
labels = y.unique()
labels = labels.sort()
conf_mat = confusion_matrix(y, predictions, labels=labels)
pd.DataFrame(conf_mat, columns=labels, index=labels)

,0,1,2,3,4,5,6,7,8,9,10,11
0,66,4,0,0,0,0,0,0,0,0,1,0
1,9,48,0,0,0,0,0,0,0,0,3,0
2,0,4,58,2,0,0,0,0,0,0,0,0
3,0,1,6,52,0,0,0,0,2,0,0,0
4,0,0,0,5,58,3,0,0,0,0,0,0
5,0,0,0,2,14,50,1,2,1,0,0,0
6,0,0,0,0,3,7,39,8,2,0,0,0
7,0,0,0,0,0,4,5,58,1,0,0,0
8,0,0,0,2,0,2,0,2,53,4,0,0
9,0,2,3,0,0,0,0,1,1,50,2,0


### Exercise
What do you conclude from the confusion matrix? Does this confusion matrix tell us anything on how well the model will perform on new data?

Answer: The model seems to do quite well, it only makes a few mistakes.
However, we predict on the same data that we trained on. To get a fair estimate of the perfomance, we need to apply it on a new data set, such as the test set

## Pipelines
Ok, suppose we want to predict on our test set. But remember, we also did a data transformation, namely scaling, in the preprocessing phase. We need to do the exact same steps for our test set!

To make it easier to reproduce all steps for new datasets, and to make sure that for each step, the train-test split is well guarded, sklearn provides pipeline. 
So we start again with the unscaled data, but now define a pipeline:

In [8]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

In [12]:
weather_train = pd.read_csv('data/weather_train.csv')
X = weather_train[features]
y = weather_train['MONTH']

In [13]:
# We give the pipeline tuples of step names, and step objects
pipe = Pipeline([
    ('scale', MinMaxScaler()),
    ('model', KNeighborsClassifier(n_neighbors=3))
])

In [14]:
pipe.fit(X, y)

Pipeline(steps=[('scale', MinMaxScaler()),
                ('model', KNeighborsClassifier(n_neighbors=3))])

To now test in on our test set: (note that we still have to drop null values)

In [17]:
weather_test = pd.read_csv('data/weather_test.csv')
X_test = weather_test[features]
y_test = weather_test['MONTH']

In [23]:
pred_test = pipe.predict(X_test)
conf_mat = confusion_matrix(y_test, pred_test, labels=labels)
pd.DataFrame(conf_mat, columns=labels, index=labels)

,0,1,2,3,4,5,6,7,8,9,10,11
0,17,3,0,0,0,0,0,0,0,0,1,1
1,7,14,2,0,0,0,0,0,0,0,2,0
2,2,4,21,0,0,0,0,0,0,2,0,0
3,0,0,6,20,3,0,0,0,0,0,0,0
4,0,0,0,6,17,3,1,0,0,0,0,0
5,0,0,0,0,7,7,2,3,1,0,0,0
6,0,0,0,0,2,7,11,12,2,0,0,0
7,0,0,0,0,1,4,2,17,1,0,0,0
8,0,0,0,1,1,0,1,5,16,3,0,0
9,1,0,6,1,0,0,0,0,2,23,0,1


We can also calculate the *accuracy*

In [24]:
from sklearn.metrics import accuracy_score

In [26]:
print('Train accuracy:', accuracy_score(y, predictions))
print('Test accuracy:', accuracy_score(y_test, pred_test))

Train accuracy: 0.8211488250652742
Test accuracy: 0.5896656534954408


So it turns out our model does not perform so well in real life afer all..
This is called 'overfitting'